# 📈 RUL Prediction Training (Transformer + Attention)

This notebook uses a **Transformer Encoder** for RUL prediction on C-MAPSS.

**Improvements over LSTM:**
- Self-attention captures long-range dependencies
- Parallel processing (faster training)
- Better at handling variable operating conditions

**Expected RMSE:** FD001 ~12-15, FD002 ~22-26

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import os
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 1. Configuration

In [ ]:
# ============= UPDATE THIS PATH =============
DATA_PATH = '/content/drive/MyDrive/MaintanenceAI/Data/CMaps'
SAVE_PATH = '/content/drive/MyDrive/MaintanenceAI/trained_models'
# =============================================

SUBSETS = ['FD001', 'FD002', 'FD003', 'FD004']

# Hyperparameters (optimized for Transformer)
SEQUENCE_LENGTH = 80        # Longer context
MAX_RUL = 125
D_MODEL = 64                # Transformer dimension
N_HEADS = 4                 # Attention heads
N_LAYERS = 3                # Transformer layers
D_FF = 128                  # Feed-forward dimension
DROPOUT = 0.1

EPOCHS = 100                # More epochs
BATCH_SIZE = 64
LEARNING_RATE = 1e-4        # Lower LR for Transformer

print(f'Training Transformer on: {SUBSETS}')

## 2. Transformer Model

In [ ]:
class PositionalEncoding(nn.Module):
    """Adds positional information to sequence."""
    def __init__(self, d_model, max_len=500, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class TransformerRUL(nn.Module):
    """Transformer Encoder for RUL Prediction with Attention."""
    
    def __init__(self, input_size, d_model=64, n_heads=4, n_layers=3, d_ff=128, dropout=0.1):
        super().__init__()
        
        # Input projection
        self.input_proj = nn.Linear(input_size, d_model)
        
        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout=dropout)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Attention pooling (learns which timesteps matter most)
        self.attention_weights = nn.Linear(d_model, 1)
        
        # Output regression
        self.regressor = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        # x: [batch, seq_len, features]
        
        # Project to d_model
        x = self.input_proj(x)
        
        # Add positional encoding
        x = self.pos_encoder(x)
        
        # Transformer encoding
        x = self.transformer(x)
        
        # Attention pooling (weighted average)
        attn_scores = torch.softmax(self.attention_weights(x), dim=1)
        x = torch.sum(x * attn_scores, dim=1)  # [batch, d_model]
        
        # Predict RUL
        return self.regressor(x)

## 3. Data Loading

In [ ]:
COLUMN_NAMES = ['unit', 'cycle', 'op1', 'op2', 'op3'] + [f'sensor_{i}' for i in range(1, 22)]
DROP_SENSORS = ['sensor_1', 'sensor_5', 'sensor_6', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']

def get_feature_columns():
    all_sensors = [f'sensor_{i}' for i in range(1, 22)]
    feature_cols = [s for s in all_sensors if s not in DROP_SENSORS]
    return ['op1', 'op2', 'op3'] + feature_cols

def load_data(subset):
    train_df = pd.read_csv(f'{DATA_PATH}/train_{subset}.txt', sep=r'\s+', header=None, names=COLUMN_NAMES)
    test_df = pd.read_csv(f'{DATA_PATH}/test_{subset}.txt', sep=r'\s+', header=None, names=COLUMN_NAMES)
    rul_true = pd.read_csv(f'{DATA_PATH}/RUL_{subset}.txt', header=None).values.flatten()
    return train_df, test_df, rul_true

def compute_rul(df, max_rul=125):
    df = df.copy()
    max_cycles = df.groupby('unit')['cycle'].max()
    df['RUL'] = df.apply(lambda row: max_cycles[row['unit']] - row['cycle'], axis=1)
    df['RUL'] = df['RUL'].clip(upper=max_rul)
    return df

def create_sequences(df, seq_len, feature_cols):
    sequences, targets = [], []
    for unit_id in df['unit'].unique():
        unit_data = df[df['unit'] == unit_id].sort_values('cycle')
        if len(unit_data) < seq_len:
            continue
        features = unit_data[feature_cols].values
        rul_values = unit_data['RUL'].values
        for i in range(len(unit_data) - seq_len + 1):
            sequences.append(features[i:i + seq_len])
            targets.append(rul_values[i + seq_len - 1])
    return np.array(sequences, dtype=np.float32), np.array(targets, dtype=np.float32)

def create_test_sequences(df, seq_len, feature_cols):
    sequences = []
    for unit_id in sorted(df['unit'].unique()):
        unit_data = df[df['unit'] == unit_id].tail(seq_len)
        if len(unit_data) < seq_len:
            pad_len = seq_len - len(unit_data)
            padding = np.zeros((pad_len, len(feature_cols)))
            seq = np.vstack([padding, unit_data[feature_cols].values])
        else:
            seq = unit_data[feature_cols].values
        sequences.append(seq)
    return np.array(sequences, dtype=np.float32)

## 4. Dataset

In [ ]:
class CMAPSSDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.tensor(sequences, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

## 5. Train All Subsets

In [ ]:
def train_subset(subset):
    print(f'\n{"="*60}')
    print(f'🚀 Training Transformer on {subset}')
    print(f'{"="*60}')
    
    # Load & preprocess
    train_df, test_df, rul_true = load_data(subset)
    train_df = compute_rul(train_df, MAX_RUL)
    feature_cols = get_feature_columns()
    
    # Normalize
    scaler = StandardScaler()
    train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])
    test_df[feature_cols] = scaler.transform(test_df[feature_cols])
    
    # Create sequences
    X_train, y_train = create_sequences(train_df, SEQUENCE_LENGTH, feature_cols)
    X_test = create_test_sequences(test_df, SEQUENCE_LENGTH, feature_cols)
    print(f'Train: {len(X_train)}, Test: {len(X_test)}')
    
    # DataLoader
    train_loader = DataLoader(CMAPSSDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
    
    # Model
    input_size = len(feature_cols)
    model = TransformerRUL(
        input_size=input_size,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        n_layers=N_LAYERS,
        d_ff=D_FF,
        dropout=DROPOUT
    ).to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    # Training
    best_rmse = float('inf')
    best_model = None
    
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        
        for seq, label in tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}', leave=False):
            seq, label = seq.to(device), label.to(device).unsqueeze(1)
            
            optimizer.zero_grad()
            output = model(seq)
            loss = criterion(output, label)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()
            
            total_loss += loss.item()
        
        scheduler.step()
        
        # Evaluate every 10 epochs
        if (epoch + 1) % 10 == 0:
            model.eval()
            with torch.no_grad():
                preds = model(torch.tensor(X_test).to(device)).cpu().numpy().flatten()
            rmse = np.sqrt(np.mean((preds - rul_true) ** 2))
            print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {total_loss/len(train_loader):.4f}, Test RMSE: {rmse:.2f}')
            
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model.state_dict().copy()
    
    # Final evaluation with best model
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        predictions = model(torch.tensor(X_test).to(device)).cpu().numpy().flatten()
    rmse = np.sqrt(np.mean((predictions - rul_true) ** 2))
    print(f'\n📊 Best Test RMSE: {rmse:.2f} cycles')
    
    # Save
    os.makedirs(SAVE_PATH, exist_ok=True)
    save_file = f'{SAVE_PATH}/rul_transformer_{subset}.pth'
    torch.save({
        'model_state_dict': best_model,
        'input_size': input_size,
        'd_model': D_MODEL,
        'n_heads': N_HEADS,
        'n_layers': N_LAYERS,
        'sequence_length': SEQUENCE_LENGTH,
        'feature_cols': feature_cols,
        'scaler_mean': scaler.mean_.tolist(),
        'scaler_scale': scaler.scale_.tolist(),
        'test_rmse': rmse
    }, save_file)
    print(f'✅ Saved: {save_file}')
    
    return {'subset': subset, 'rmse': rmse, 'predictions': predictions, 'true': rul_true}

# Train all
all_results = []
for subset in SUBSETS:
    try:
        results = train_subset(subset)
        all_results.append(results)
    except FileNotFoundError:
        print(f'⚠️ {subset} not found, skipping...')

print(f'\n{"="*60}')
print('🎉 All training complete!')
print(f'{"="*60}')

## 6. Results Summary

In [ ]:
print('\n📋 Final Results (Transformer):\n')
print(f'{"Subset":<10} {"Test RMSE":<15} {"Status":<15}')
print('-' * 45)

for r in all_results:
    if r['rmse'] < 15:
        status = '🏆 Excellent'
    elif r['rmse'] < 20:
        status = '✅ Good'
    elif r['rmse'] < 28:
        status = '⚠️ Acceptable'
    else:
        status = '❌ Needs work'
    print(f'{r["subset"]:<10} {r["rmse"]:<15.2f} {status:<15}')

avg_rmse = np.mean([r['rmse'] for r in all_results])
print(f'\nAverage RMSE: {avg_rmse:.2f} cycles')

## 7. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, r in enumerate(all_results):
    ax = axes[idx]
    ax.scatter(r['true'], r['predictions'], alpha=0.5, s=20)
    ax.plot([0, MAX_RUL], [0, MAX_RUL], 'r--', label='Perfect')
    ax.set_xlabel('Actual RUL')
    ax.set_ylabel('Predicted RUL')
    ax.set_title(f'{r["subset"]} (RMSE: {r["rmse"]:.2f})')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('transformer_rul_results.png', dpi=150)
plt.show()